In [1]:
import scanpy as sc
import scrublet as scr
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
from gprofiler import gprofiler

import rpy2.rinterface_lib.callbacks
import logging

from rpy2.robjects import pandas2ri
import anndata2ri

/home/elismail/.local/lib/python3.6/site-packages/numba/core/errors.py:154: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [2]:
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
#rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

#plt.rcParams['figure.figsize']=(8,8) #rescale figures
#sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
#sc.logging.print_versions()

In [3]:
%%R

library(scran)
library(Matrix)
library(dplyr)
library(tidyr)
library(RColorBrewer)
library(slingshot)
library(monocle)
library(gam)
library(clusterExperiment)
library(ggplot2)
library(plyr)
library(MAST)


R[write to console]: Lade nötiges Paket: SingleCellExperiment

R[write to console]: Lade nötiges Paket: SummarizedExperiment

R[write to console]: Lade nötiges Paket: MatrixGenerics

R[write to console]: Lade nötiges Paket: matrixStats

R[write to console]: 
Attache Paket: ‘MatrixGenerics’


R[write to console]: The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiff

In [4]:
# Set up data loading
#Data files
sample_strings = ['1820', '1822']
file_base = '/home/elismail/Dokumente/scRNA-Data/scRNA_Tutorial/'
exp_string = '_Expression_'
data_file_end = 'Data.st'

In [5]:
#First data set load & annotation
#Parse Filenames
sample = sample_strings.pop(0)
data_file = file_base+sample+exp_string+data_file_end

#Load data
adata_1820 = pd.read_csv(data_file, sep='\t')

In [6]:
adata_1820.drop(['RSEC_Reads', 'RSEC_Adjusted_Molecules'], axis=1, inplace=True)
#print('Counts matrix shape: {} rows, {} columns'.format(adata_1820.shape[0], adata_1820.shape[1]))

In [7]:
cells = adata_1820["Cell_Index"].unique()
genes = adata_1820["Gene"].unique()

In [8]:
cells_dict = {i: index for index, i in enumerate(cells)}
genes_dict = {i: index for index, i in enumerate(genes)}
dense_matrix = np.zeros(shape=(len(cells), len(genes)), dtype=np.int16())

In [9]:
size = int(adata_1820.shape[0]/50)
size

214516

In [10]:
for index, (cell, gene, value) in enumerate(zip(adata_1820["Cell_Index"].tolist(), adata_1820["Gene"].tolist(), adata_1820["Raw_Molecules"].tolist())):
    if index % size ==0 and index != 0:
        print("#", end="")
    dense_matrix[cells_dict.get(cell, None), genes_dict.get(gene, None)] = value

##################################################

In [11]:
np.count_nonzero(dense_matrix)

10725802

In [12]:
214516*50

10725800

In [7]:
#Loop to load the rest of data sets
for i in range(len(sample_strings)):
    #Parse Filenames
    sample = sample_strings[i]
    data_file = file_base+sample+exp_string+data_file_end

    #Load data
    adata_1822 = sc.read(data_file, sep='\t', cache=True)

In [8]:
print('Counts matrix shape: {} rows, {} columns'.format(adata_1822.shape[0], adata_1822.shape[1]))


Counts matrix shape: 12512 rows, 32675 columns
